### Preparing data for clustering

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pathlib import Path

In [19]:
file_path = Path(r'..\data\03_processed\01_ready_data.csv')
df = pd.read_csv(file_path)

In [25]:
cols_for_grouping = df.loc[:, ~df.columns.isin(['Nazwa', 'ID', 'Adres e-mail'])].columns

In [28]:
df[cols_for_grouping] = StandardScaler().fit_transform(df[cols_for_grouping])
df.head()

,ID,Adres e-mail,Nazwa,W jaki sposób chcesz uczestniczyć w Community?,R,Python,Bash,GIT,Docker,CLI,...,UEWAdmin,FinTech,HealthTech,FashionTech,Ecommerce,SportTech,NonProfit,PropTech,Cybersecurity,HR
0,1,anonymous,NaN,1.207615,0.146891,0.587995,1.433259,1.624204,1.333848,2.097953,...,0.618718,1.134202,0.544016,2.142155,1.437744,1.930591,0.671218,1.952102,0.722897,1.052970
1,2,anonymous,NaN,-0.828079,1.109844,0.587995,-0.257987,0.818927,0.514087,0.447563,...,-0.424264,0.253703,0.544016,1.043136,0.390498,0.845783,0.671218,0.833908,0.722897,1.052970
2,3,anonymous,NaN,-0.828079,-1.779014,-2.302981,-1.103610,-0.791629,-1.125434,-1.202827,...,0.618718,1.134202,-1.462044,-1.154901,0.390498,-1.323834,-0.344214,0.833908,-1.646600,-1.247964
3,4,anonymous,NaN,1.207615,1.109844,0.587995,0.587636,0.013649,0.514087,0.447563,...,1.661701,1.134202,0.544016,1.043136,1.437744,-0.239026,1.686650,-0.284287,0.722897,-0.097497
4,5,anonymous,NaN,1.207615,1.109844,0.587995,0.587636,0.013649,0.514087,0.447563,...,1.661701,1.134202,1.547046,-0.055882,-0.656747,-0.239026,0.671218,-0.284287,-0.461851,1.052970
